In [137]:
import pandas as pd
import parser
import imp

pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 100
imp.reload(parser)

<module 'parser' from '/home/tonko22/PycharmProjects/npo_classification/parser.py'>

In [139]:
!git commit -a -m "minor fix"

[master 9e5d6f8] minor fix
 2 files changed, 8 insertions(+), 61 deletions(-)


In [154]:
dataset = pd.read_excel("np_ngo_classification.xlsx")
dataset.head(1)

,Регион,Название организации,ИНН,Год регистрации,Адресс,Возможность пожертвовать деньги на сайте,Возможность обратиться за помощью,Ссылка на сайт,Ссылка на соц сеть,Вид деятельности 1,...,Вид деятельности 3,Вид деятельности 4,Вид деятельности 5,Вид деятельности 6,Вид деятельности 7,Вид деятельности 8,Вид деятельности 9,Вид деятельности 10,Вид деятельности 11,Вид деятельности 12
0,Москва,"Межрегиональная общественная благотворительная организация ""Общество защиты прав потребителей и ...",5032173157,2007-07-12,"г Москва, Орехово-Борисово Южное р-н, МКАД 23-й км, д 6",0,0,"http://greenstartup.ru/, http:/9911850.ru/",https://princip1.livejournal.com/,3_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
# Drop empty rows
sites = dataset[~dataset["Ссылка на сайт"].isnull()]
sites.drop(columns=["Ссылка на соц сеть"], inplace=True)
social = dataset[~dataset["Ссылка на соц сеть"].isnull()]

# Merging social web-page to a web-site, adding new data samples 
social["Ссылка на сайт"] = social["Ссылка на соц сеть"]
social.drop(columns=["Ссылка на соц сеть"], inplace=True)
concat_dataset = pd.concat([sites, social], ignore_index=True)

/home/tonko22/.virtualenvs/semantic_env_36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/tonko22/.virtualenvs/semantic_env_36/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/tonko22/.virtualenvs/semantic_env_36/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

In [156]:
concat_dataset["Ссылка на сайт"] = concat_dataset["Ссылка на сайт"].apply(parser.link_splitter)
dataset.dropna(axis=0, subset=["Ссылка на сайт"], inplace=True)
concat_dataset["num_sites"] = concat_dataset["Ссылка на сайт"].apply(len)

In [159]:
# ! WARNING: This cell needs to be executed 3 times!
# TODO: 3 times-loop
max_len = len(concat_dataset)
for index, row in concat_dataset.iterrows():
    if len(row["Ссылка на сайт"])>1:
        sites = row["Ссылка на сайт"]
        last_site = sites.pop(-1)
        concat_dataset["Ссылка на сайт"].loc[index] = sites
        row["Ссылка на сайт"] = [last_site]
        concat_dataset.loc[max_len] = row
        max_len += 1

/home/tonko22/.virtualenvs/semantic_env_36/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [161]:
concat_dataset["Ссылка на сайт"] = concat_dataset["Ссылка на сайт"].apply(lambda lst: lst[0].strip())

In [163]:
# Fixing mistakes
concat_dataset["Ссылка на сайт"].loc[62] = "http://арктика-надежда.рф" 
concat_dataset["Ссылка на сайт"].loc[154] = "http://9911850.ru/"

/home/tonko22/.virtualenvs/semantic_env_36/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [165]:
concat_dataset["web_text"] = concat_dataset["Ссылка на сайт"].apply(parser.get_text_contents)

/home/tonko22/PycharmProjects/npo_classification/parser.py:45: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 45 of the file /home/tonko22/PycharmProjects/npo_classification/parser.py. To get rid of this warning, pass the additional argument 'features="html5lib"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)
/home/tonko22/PycharmProjects/npo_classification/parser.py:59: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this war

In [171]:
concat_dataset = concat_dataset.dropna(subset=["web_text"])
concat_dataset = concat_dataset[concat_dataset['web_text'].astype(str) != '[]']
concat_dataset = concat_dataset.reset_index().drop("index", axis=1)

In [174]:
concat_dataset["web_text"] = concat_dataset["web_text"].apply(lambda x: " ".join(x))

In [175]:
concat_dataset["web_text"]

0      function set_cookie(){var now = new Date();var time = now.getTime();time += 19360000 * 1000;now....
1      html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-tra...
2      html Благотворительный фонд Столица милосердия [if IE 6]>\n<link rel='stylesheet' type='text/css...
3      html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dt...
4      html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd" Благ...
5      html Контактная информация и реквизиты | Фонд Спивакова [if lt IE 9]>\n<script src="http://spiva...
6      html Главная | Я есть <script src="http://ajax.googleapis.com/ajax/libs/jquery/1.12.1/jquery.min...
7      html Фонд «Ковчег» | Поможем вместе window._wpemojiSettings = {"baseUrl":"http:\/\/s.w.org\/imag...
8      html PUBLIC "-//W3C//DTD XHTML 1.1//EN" "http://www.w3.org/TR/xhtml11/DTD/xhtml11.dtd" Áëàãîòâîð...
9      html if(!window.BX)window.BX={

In [176]:
concat_dataset.to_pickle(path="np_ngo_classification_links.pkl")

In [177]:
concat_dataset["web_text"].loc[22]

'html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd" if(!window.BX)window.BX={message:function(mess){if(typeof mess==\'object\') for(var i in mess) BX.message[i]=mess[i]; return true;}}; Ак барс созидание <!--\nfunction MM_swapImgRestore() { //v3.0\n  var i,x,a=document.MM_sr; for(i=0;a&&i<a.length&&(x=a[i])&&x.oSrc;i++) x.src=x.oSrc;\n}\n\nfunction MM_preloadImages() { //v3.0\n  var d=document; if(d.images){ if(!d.MM_p) d.MM_p=new Array();\n    var i,j=d.MM_p.length,a=MM_preloadImages.arguments; for(i=0; i<a.length; i++)\n    if (a[i].indexOf("#")!=0){ d.MM_p[j]=new Image; d.MM_p[j++].src=a[i];}}\n}\n\nfunction MM_findObj(n, d) { //v4.01\n  var p,i,x;  if(!d) d=document; if((p=n.indexOf("?"))>0&&parent.frames.length) {\n    d=parent.frames[n.substring(p+1)].document; n=n.substring(0,p);}\n  if(!(x=d[n])&&d.all) x=d.all[n]; for (i=0;!x&&i<d.forms.length;i++) x=d.forms[i][n];\n  for(i=0;!x&&d.layers&&i<d.layers.length;i++) x=MM_

In [6]:
mapping = pd.read_excel("np_ngo_mappings.xlsx")
mapping

In [7]:
dataset["links"] = dataset["Ссылка на сайт"].apply(parser.get_links)

/home/ubuntu/PycharmProjects/nko_classification/parser.py:16: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 16 of the file /home/ubuntu/PycharmProjects/nko_classification/parser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)
